In [70]:
import pandas
import numpy as np
import matplotlib.pyplot as plt
import torch
from pprint import pprint

import connect_plot
import connect_four

In [186]:
pairs = connect_four.get_pairs()
p0 = [p[0] for p in pairs]
p1 = [p[1] for p in pairs]

In [67]:
df_games1 = pandas.read_csv("data/connect-4.data", header=None)
print(len(df_games1))
cols = [f"a{i}" for i in range(1, 7)] \
    + [f"b{i}" for i in range(1, 7)] \
    + [f"c{i}" for i in range(1, 7)] \
    + [f"d{i}" for i in range(1, 7)] \
    + [f"e{i}" for i in range(1, 7)] \
    + [f"f{i}" for i in range(1, 7)] \
    + [f"g{i}" for i in range(1, 7)]
df_games1 = df_games1.rename(columns={i: c for i, c in enumerate(cols)})
df_games1 = df_games1.rename(columns={42: "result"})
for c in cols:
    df_games1[c] = df_games1[c].map({"b": 0, "x": 1, "o": -1})
df_games1["result"] = df_games1["result"].map({"win": 1, "draw": 0, "loss": -1})

vals_x1 = np.flip(np.swapaxes(np.reshape(df_games1.iloc[:,:42].values, (67557, 7, 6)), 1, 2), axis=1).astype(np.float64)
vals_y1 = df_games1["result"].values
# df_games1.head(3)

67557


In [279]:
np.save("data/move8_boards", vals_x1)
np.save("data/move8_winner", vals_y1)

In [244]:
df_games2 = pandas.read_csv("data/c4_game_database.csv")
print(f"Initial dataset::{len(df_games2)}")
df_games2 = df_games2[~df_games2["winner"].isnull()]
print(f"Remove nulls::{len(df_games2)}")
vals_x2 = np.reshape(df_games2.iloc[:,:42].values, (len(df_games2), 6, 7))
vals_y2 = df_games2["winner"].values

376619


In [245]:
mod_boards = vals_x2 * np.reshape(vals_y2, (len(df_games2), 1, 1))
print(np.isnan(mod_boards).any())
not_zero = np.where(vals_y2 != 0)[0]
mod_boards = mod_boards[not_zero]
print(np.isnan(mod_boards).any())
# mod_indexes = vals_y2[not_zero]
print(mod_boards.shape)

vals = []
for val in np.reshape(np.int64(mod_boards + 1), (mod_boards.shape[0], 42)).astype(str):
    vals.append(''.join(val))
print(len(vals), len(set(vals)))

unique_boards = pandas.Series(vals).drop_duplicates().index.tolist()
print(len(unique_boards))
print(mod_boards[unique_boards].shape)
# print(mod_indexes[unique_boards].shape)

False
False
(362122, 6, 7)
362122 359482
359482
(359482, 6, 7)


In [259]:
win_boards = mod_boards[unique_boards]
print(win_boards.shape)

win_patterns = win_boards[:,p0,p1]
win_patterns_sum = np.sum(win_patterns, axis=2)

all_new_win_coords = {}
for board, win_pair in zip(*np.where(win_patterns_sum == 4)):
    win_coords = {(a, b) for a, b in zip(p0[win_pair], p1[win_pair])}
    new_coords = {(a, b) for a, b in zip(np.int32(6 - np.abs(win_boards[board]).sum(axis=0)), range(7))}
    new_win_coords = win_coords & new_coords
    if board in all_new_win_coords:
        all_new_win_coords[board] = all_new_win_coords[board] | new_win_coords
    else:
        all_new_win_coords[board] = new_win_coords
print(len(all_new_win_coords))

(359482, 6, 7)
359482


In [260]:
final_boards = []
final_coords = []
for board, lose_pairs in all_new_win_coords.items():
    for lpair in lose_pairs:
        board_data = win_boards[board].copy()
        board_data[lpair[0],lpair[1]] = 0
        final_boards.append(board_data)
        final_coords.append(lpair)

In [274]:
print(len(final_boards))
np.save("data/finalmove_boards", np.stack(final_boards))
np.save("data/finalmove_moves", np.stack(final_coords))

658973
